In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from numpy import shape
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from torch.autograd import Variable
from tqdm import tqdm_notebook
import math
from time import time

#### csv파일 호출

file_name='{}.csv'
path = "C:/Users/user/Desktop/SKI_VDU/Yield_model/"

X=pd.read_csv(path+file_name.format('A_VDU_UCO_BP'))
Y=pd.read_csv(path+file_name.format('A_VDU_yield'))



####PCA를 이용하여 X data matrix 차원감소 (표준화 OFF)
                                     
#X_PCA_df = StandardScaler().fit_transform(X_PCA_df)
ncomp=20
pca=PCA(n_components=ncomp)
X_PCA = pca.fit(X)
pca_var_list=list(pca.explained_variance_ratio_)
pca_varcum_list = np.cumsum(pca_var_list)
pca_num_list = list(range(1,1+len(pca_var_list)))
print(pca_varcum_list)

fig = plt.figure(figsize=(30,10))
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(pca_num_list,pca_varcum_list)

X_reduced = pca.fit_transform(X)
X_reduced_df = pd.DataFrame(X_reduced)
#X_reduced_train = X_reduced_df - X_reduced_test
#X_reduced_test = X_reduced_df.sample(n=5)

#X_reduced_train = X_reduced[0:60,:]
#X_reduced_test = X_reduced[60:62,:]

#print(shape(X_reduced_train))
#print(type(X_reduced_train))
#print(shape(X_reduced_test))
#print(type(X_reduced_test))
print(shape(X))
print(type(X))
print(len(X))
print((shape(X_reduced)))
#print(shape(X_reduced_train))
#print((shape(X_reduced_test)))



[0.75062347 0.95222209 0.97165522 0.98120588 0.98764371 0.990934
 0.99350646 0.99518999 0.99636676 0.99742103 0.99813043 0.99859519
 0.9989677  0.99919253 0.9993545  0.99950329 0.99961279 0.99971838
 0.99980077 0.99985788]
(86, 101)
<class 'pandas.core.frame.DataFrame'>
86
(86, 20)


In [2]:
#### X matrix, Y matrix 행으로 붙이기
X_reduced_array = X_reduced_df.values
X_array = X.values
Y_array = Y.values

data_num = len(X)
X_var_num = len(X_array[1])
X_reduced_var_num = len(X_reduced_array[1])
Y_var_num = len(Y_array[1])

print(data_num)
print(X_var_num)
print(Y_var_num)
print(X_reduced_var_num)

86
101
3
20


In [3]:
combXY_train = np.zeros((data_num,X_reduced_var_num+Y_var_num),dtype=float)
print(combXY_train)
combXY_train[:,:-Y_var_num] = X_reduced_array
print(X_reduced_array)
print(combXY_train)
combXY_train[:,X_reduced_var_num:X_reduced_var_num+Y_var_num] = Y_array[:,0:Y_var_num]
print(combXY_train)

#combXY_test = np.zeros((2,24), dtype=float)
#combXY_test[:,:-4] = X_reduced_test
#combXY_test[:,20:24] = testY_array[:,0:4]

#print(pd.DataFrame(combXY_train))
#print(pd.DataFrame(combXY_test))
#print(pd.DataFrame(testY_array))
#print(type(combXY_train))


#### Array를 torch로 자료형 변환

train_data = torch.from_numpy(combXY_train).float()
#test_data = torch.from_numpy(combXY_test).float()
print(shape(train_data))
#print(shape(test_data))
train_data.shape[0]


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[-6.00829186e-01  1.43628087e-02  5.57604668e-02 ... -6.00112765e-03
  -4.95348281e-03 -4.75904266e-03]
 [-5.36888602e-01  2.54835397e-01 -1.86201167e-02 ...  1.49981194e-02
   2.45057668e-02  8.83606795e-03]
 [-7.25657292e-01  5.34685406e-02  1.63360916e-01 ...  4.61268102e-03
  -1.36014426e-02 -1.11706433e-02]
 ...
 [ 4.00350423e+00  1.84141326e-01 -6.14079858e-01 ...  2.85639864e-02
  -4.42051218e-03 -3.87299196e-03]
 [ 2.37651286e+00 -1.28341972e-01 -5.87709331e-01 ...  8.65239164e-03
   4.96798742e-03  1.72423246e-03]
 [ 4.00350422e+00  1.84141311e-01 -6.14079857e-01 ...  2.85639789e-02
  -4.42050816e-03 -3.87298927e-03]]
[[-0.60082919  0.01436281  0.05576047 ...  0.          0.
   0.        ]
 [-0.5368886   0.2548354  -0.01862012 ...  0.          0.
   0.        ]
 [-0.72565729  0.05346854  0.16336092 ...  0.          0.
   0

86

In [4]:
#### Data indexing
class AVDU_dataset(Dataset):
    def __init__(self):
        self.len=train_data.shape[0]
        self.x_data_train = train_data[:,:-Y_var_num]
        self.y_data_train = train_data[:,X_reduced_var_num:X_reduced_var_num+Y_var_num]
    def __getitem__(self, index):
        return self.x_data_train[index], self.y_data_train[index]
    def __len__(self):
        return self.len
    
dataset= AVDU_dataset()


#class FTIRtestdataset(Dataset):
#    def __init__(self):
#        self.len=test_data.shape[0]
#        self.x_data_test = test_data[:,:-4]
#        self.y_data_test = test_data[:,20:24] 
#    def __getitem__(self, index):
#        return self.x_data_test[index], self.y_data_test[index]
#    def __len__(self):
#        return self.len
#test_dataset = FTIRtestdataset()

In [5]:
print(dataset)
print(type(dataset))
print(dataset.x_data_train.shape)
print(dataset.y_data_train.shape)

<class '__main__.AVDU_dataset'>
torch.Size([86, 20])
torch.Size([86, 3])


In [6]:
## Validation set 구축 및 Dataloader에 data 담기

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [56, 13, 17])
print(len(train_dataset), len(val_dataset), len(test_dataset))
print("train dataset type :",type(train_dataset))

batch_size = 1
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("train loader type :",type(train_loader))
print('Train loader에 들어간 FTIR data : %a, Label : %a' %(list(train_loader)[0][0],list(train_loader)[0][1]))

56 13 17
train dataset type : <class 'torch.utils.data.dataset.Subset'>
train loader type : <class 'torch.utils.data.dataloader.DataLoader'>
Train loader에 들어간 FTIR data : tensor([[-0.2447, -0.2757, -0.2621, -0.0684, -0.1972,  0.1167, -0.0084, -0.0815,
          0.0024, -0.0735,  0.0401, -0.0495, -0.0560, -0.0230,  0.0326,  0.0090,
          0.0005,  0.0104,  0.0056,  0.0040]]), Label : tensor([[0.3664, 0.2512, 0.3824]])


In [7]:
#for x,y in train_loader:
#    print("x size =", x.shape,"   ", "y size =", y.shape,"   ","x type =", type(x),"   ","y type =", type(y),"   ",)
    
#for x,y in val_loader:
#    print(x.shape)

In [8]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)


class LinearModel(nn.Module):
    def __init__(self): 
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(in_features=7, out_features=3, bias=True)
    
    def forward(self, x):
        x = self.linear(x)
        return x
        
    
class MLPModel(nn.Module):
    def __init__(self): 
        super(MLPModel, self).__init__()
        self.linear1 = nn.Linear(in_features=X_reduced_var_num, out_features=100, bias = True)
        self.linear2 = nn.Linear(in_features=100, out_features=50, bias = True)
        self.linear3 = nn.Linear(in_features=50,out_features=Y_var_num)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.linear3(x)
        return x

In [9]:
import torch.optim as optim
from sklearn.metrics import accuracy_score

# ====== Construct Model ====== #
#model = LinearModel()
model = MLPModel()
print('Number of {} parameters'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

if torch.cuda.is_available():
    model.cuda()

# ===== Construct Optimizer ====== #
lr = 0.005
optimizer = optim.Adam(model.parameters(), lr=lr) 

print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
    
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

    
    
list_epoch = [] 
list_train_loss = []
list_val_loss = []
epoch = 150
criterion = nn.MSELoss()

Number of 7303 parameters
Model's state_dict:
linear1.weight 	 torch.Size([100, 20])
linear1.bias 	 torch.Size([100])
linear2.weight 	 torch.Size([50, 100])
linear2.bias 	 torch.Size([50])
linear3.weight 	 torch.Size([3, 50])
linear3.bias 	 torch.Size([3])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.005, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'params': [243320216, 243321096, 243320536, 243320776, 243319816, 243321016]}]


In [10]:
for i in tqdm_notebook(range(epoch)):
    
    # ====== Train ====== #
    train_loss = 0
    model.train() 
    #optimizer.zero_grad() 

    for input_X, true_y in tqdm_notebook(train_loader):
        #print(input_X.size())
        input_X = to_var(input_X.squeeze())
        true_y=to_var(true_y).squeeze()
        #print(input_X.size())
        print(true_y)
        
        optimizer.zero_grad() 
        pred_y = model(input_X).squeeze()
        print(pred_y)
        loss = criterion(pred_y, true_y)
        #loss = criterion(pred_y.squeeze(), torch.max(true_y.long(), 1)[1])
        loss.backward() 
        optimizer.step() 
        train_loss += loss.cpu().detach().numpy()
    train_loss = train_loss / len(train_loader)
    list_train_loss.append(train_loss)
    list_epoch.append(i)
    
    
    # ====== Validation ====== #
    val_loss = 0
    model.eval()
    #optimizer.zero_grad()
    
    for input_X, true_y in tqdm_notebook(val_loader):
        input_X = to_var(input_X)
        optimizer.zero_grad()
        true_y=to_var(true_y).squeeze()
        pred_y = model(input_X).squeeze()
        loss = criterion(pred_y, true_y)
        #loss = criterion(pred_y.squeeze(), torch.max(true_y.long(), 1)[1])
        val_loss += loss.cpu().detach().numpy()
    val_loss = val_loss / len(val_loader)
    list_val_loss.append(val_loss)

    

tensor([0.3729, 0.3104, 0.3167], device='cuda:0')
tensor([ 0.0692, -0.0180,  0.0627], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
tensor([0.4284, 0.2301, 0.3415], device='cuda:0')
tensor([0.0856, 0.0269, 0.1056], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4230, 0.2746, 0.3023], device='cuda:0')
tensor([0.2880, 0.1353, 0.1996], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3914, 0.2676, 0.3411], device='cuda:0')
tensor([0.2344, 0.1199, 0.1779], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4780, 0.2515, 0.2705], device='cuda:0')
tensor([0.8192, 0.5058, 0.4992], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3741, 0.2886, 0.3373], device='cuda:0')
tensor([0.3373, 0.2589, 0.2739], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3788, 0.2846, 0.3366], device='cuda:0')
tensor([0.2786, 0.2151, 0.2123], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3937, 0.2276, 0.3787], device='cuda:0')
tensor([0.3367, 0.2626, 0.2876], device='cud

tensor([0.3696, 0.3364, 0.2940], device='cuda:0')
tensor([0.4251, 0.2415, 0.4970], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4273, 0.2470, 0.3257], device='cuda:0')
tensor([0.3614, 0.2464, 0.4045], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4103, 0.2276, 0.3621], device='cuda:0')
tensor([0.4276, 0.2615, 0.4640], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3607, 0.2923, 0.3470], device='cuda:0')
tensor([0.3683, 0.2532, 0.3223], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3778, 0.2629, 0.3593], device='cuda:0')
tensor([0.3455, 0.2592, 0.3482], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3848, 0.2191, 0.3961], device='cuda:0')
tensor([0.3315, 0.2390, 0.3188], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3549, 0.2698, 0.3753], device='cuda:0')
tensor([0.3496, 0.2517, 0.3085], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3968, 0.2350, 0.3682], device='cuda:0')
tensor([0.3602, 0.2588, 0.3034], device='cuda:0', grad

tensor([0.3778, 0.2629, 0.3593], device='cuda:0')
tensor([0.4392, 0.3212, 0.4023], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3914, 0.2676, 0.3411], device='cuda:0')
tensor([0.3776, 0.2488, 0.3215], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3504, 0.2764, 0.3732], device='cuda:0')
tensor([0.3755, 0.2589, 0.3848], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3788, 0.2846, 0.3366], device='cuda:0')
tensor([0.4124, 0.2672, 0.3910], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3796, 0.2330, 0.3874], device='cuda:0')
tensor([0.3718, 0.2371, 0.3962], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3878, 0.2163, 0.3959], device='cuda:0')
tensor([0.3632, 0.2202, 0.3880], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4149, 0.2332, 0.3519], device='cuda:0')
tensor([0.3267, 0.2260, 0.3244], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3622, 0.2516, 0.3861], device='cuda:0')
tensor([0.3430, 0.2252, 0.3750], device='cuda:0', grad

tensor([0.3602, 0.2552, 0.3847], device='cuda:0')
tensor([0.3895, 0.2443, 0.3624], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3811, 0.2663, 0.3526], device='cuda:0')
tensor([0.4101, 0.2492, 0.3463], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3788, 0.2846, 0.3366], device='cuda:0')
tensor([0.3860, 0.2249, 0.3741], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3532, 0.2769, 0.3698], device='cuda:0')
tensor([0.3741, 0.2247, 0.3667], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3960, 0.2298, 0.3742], device='cuda:0')
tensor([0.3215, 0.1947, 0.3608], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3709, 0.3104, 0.3187], device='cuda:0')
tensor([0.3646, 0.2436, 0.3625], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4295, 0.2518, 0.3187], device='cuda:0')
tensor([0.3779, 0.2642, 0.3416], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4948, 0.2410, 0.2642], device='cuda:0')
tensor([0.5419, 0.2717, 0.2625], device='cuda:0', grad

tensor([0.5235, 0.2122, 0.2643], device='cuda:0')
tensor([0.4820, 0.2446, 0.2738], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4043, 0.2765, 0.3192], device='cuda:0')
tensor([0.4030, 0.2564, 0.3260], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3865, 0.2218, 0.3916], device='cuda:0')
tensor([0.3866, 0.2025, 0.4007], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4374, 0.2373, 0.3253], device='cuda:0')
tensor([0.4041, 0.2492, 0.3363], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3549, 0.2698, 0.3753], device='cuda:0')
tensor([0.4056, 0.2554, 0.3357], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4607, 0.3025, 0.2368], device='cuda:0')
tensor([0.5303, 0.3103, 0.2096], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3796, 0.2330, 0.3874], device='cuda:0')
tensor([0.3730, 0.2337, 0.3743], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3765, 0.2898, 0.3338], device='cuda:0')
tensor([0.4192, 0.2710, 0.2990], device='cuda:0', grad

...

tensor([0.3679, 0.3310, 0.3011], device='cuda:0')
tensor([0.4252, 0.2558, 0.3262], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3410, 0.2952, 0.3638], device='cuda:0')
tensor([0.4052, 0.2657, 0.3356], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4295, 0.2518, 0.3187], device='cuda:0')
tensor([0.4132, 0.2744, 0.3201], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3988, 0.2480, 0.3532], device='cuda:0')
tensor([0.4107, 0.2647, 0.3287], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3504, 0.2764, 0.3732], device='cuda:0')
tensor([0.3763, 0.2640, 0.3633], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3960, 0.2298, 0.3742], device='cuda:0')
tensor([0.3988, 0.2360, 0.3410], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4069, 0.2792, 0.3139], device='cuda:0')
tensor([0.3849, 0.2880, 0.3274], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3788, 0.2846, 0.3366], device='cuda:0')
tensor([0.3654, 0.2839, 0.3591], device='cuda:0', grad

tensor([0.4149, 0.2332, 0.3519], device='cuda:0')
tensor([0.3958, 0.2491, 0.3414], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3968, 0.2350, 0.3682], device='cuda:0')
tensor([0.3863, 0.2563, 0.3395], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3664, 0.2512, 0.3824], device='cuda:0')
tensor([0.3859, 0.2541, 0.3457], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3729, 0.3104, 0.3167], device='cuda:0')
tensor([0.4118, 0.3024, 0.3062], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3709, 0.3104, 0.3187], device='cuda:0')
tensor([0.3859, 0.2531, 0.3658], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3904, 0.2322, 0.3774], device='cuda:0')
tensor([0.4226, 0.2093, 0.4083], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3960, 0.2298, 0.3742], device='cuda:0')
tensor([0.4016, 0.2550, 0.3444], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3532, 0.2769, 0.3698], device='cuda:0')
tensor([0.3834, 0.2694, 0.3628], device='cuda:0', grad

tensor([0.3679, 0.3310, 0.3011], device='cuda:0')
tensor([0.3925, 0.2727, 0.3437], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4828, 0.2055, 0.3117], device='cuda:0')
tensor([0.4416, 0.1776, 0.3357], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4149, 0.2332, 0.3519], device='cuda:0')
tensor([0.3902, 0.2686, 0.3537], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3865, 0.2218, 0.3916], device='cuda:0')
tensor([0.4022, 0.2415, 0.3877], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3811, 0.2663, 0.3526], device='cuda:0')
tensor([0.3820, 0.2974, 0.3385], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3741, 0.2886, 0.3373], device='cuda:0')
tensor([0.3739, 0.3024, 0.3393], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4607, 0.3025, 0.2368], device='cuda:0')
tensor([0.4043, 0.2332, 0.2996], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3937, 0.2276, 0.3787], device='cuda:0')
tensor([0.3763, 0.2578, 0.3661], device='cuda:0', grad

tensor([0.3880, 0.2284, 0.3835], device='cuda:0')
tensor([0.3851, 0.2101, 0.3846], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3988, 0.2480, 0.3532], device='cuda:0')
tensor([0.3772, 0.2390, 0.3478], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3856, 0.2382, 0.3761], device='cuda:0')
tensor([0.3727, 0.2272, 0.3859], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3922, 0.2368, 0.3710], device='cuda:0')
tensor([0.3732, 0.2377, 0.3663], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.3602, 0.2552, 0.3847], device='cuda:0')
tensor([0.3561, 0.2324, 0.4045], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4284, 0.2301, 0.3415], device='cuda:0')
tensor([0.3831, 0.2297, 0.3668], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.5023, 0.2480, 0.2497], device='cuda:0')
tensor([0.5061, 0.1871, 0.3201], device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0.4828, 0.2055, 0.3117], device='cuda:0')
tensor([0.4914, 0.1764, 0.3290], device='cuda:0', grad

KeyboardInterrupt: 

In [ ]:
list_D70_error=[]
list_D100_error=[]
list_D150_error=[]
list_acc=[]

for input_X, true_y in tqdm_notebook(test_loader):
    input_X = to_var(input_X)
    pred_y = model(input_X)
    pred_y = pred_y.cpu()
    true_y=true_y.squeeze().numpy()
    pred_y=pred_y.detach().numpy()[0]


    D70_error = round(abs(((true_y[0]-pred_y[0])/true_y[0])*100),3)
    D100_error = round(abs(((true_y[1]-pred_y[1])/true_y[1])*100),3)
    D150_error = round(abs(((true_y[2]-pred_y[2])/true_y[2])*100),3)
    acc = round((D70_error+D100_error+D150_error)/3,3)
    
    list_D70_error.append(D70_error)
    list_D100_error.append(D100_error)
    list_D150_error.append(D150_error)
    list_acc.append(acc)
    



In [ ]:
print("Testset error for 70D :", list_D70_error)
print("Testset error for 100D :", list_D100_error)
print("Testset error for 150D :", list_D150_error)

print('Avg error for 70D :', sum(list_D70_error)/len(list_D70_error))
print('Avg error for 100D :', sum(list_D100_error)/len(list_D100_error))
print('Avg error for 150D :', sum(list_D150_error)/len(list_D150_error))

In [ ]:
##loss 저장
#pdtrainloss = pd.DataFrame(list_train_loss)
#pdvalloss = pd.DataFrame(list_val_loss)
#pdtrainloss.to_csv("trainloss0.01.csv",header = False, index=False)
#pdvalloss.to_csv("valloss0.01.csv",header = False, index=False)

In [ ]:
fig = plt.figure(figsize=(30,10))

# ====== Loss Fluctuation ====== #
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(list_epoch, list_train_loss, label='train_loss')
ax1.plot(list_epoch, list_val_loss, '--', label='val_loss')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')
ax1.grid()
ax1.legend()
ax1.set_title('epoch vs loss')

# ====== Metric Fluctuation ====== #
#ax2 = fig.add_subplot(1, 2, 2)
#ax2.plot(list_acc_epoch, list_acc, marker='x', label='Accuracy metric')
#ax2.set_xlabel('epoch')
#ax2.set_ylabel('Acc')
#ax2.grid()
#ax2.legend()
#ax2.set_title('epoch vs Accuracy')

#plt.show()